In [ ]:
from utils.RE_utils.CERED.RE_model_base import RelationExtractionModel
from utils.RE_utils.CERED.RE_model_base import consts
from transformers import AutoTokenizer
import torch
import json

labels_path = "/home/stulcrad/ner-radek-stulc/utils/RE_utils/CERED/CERED2_LABELS"
id2label = {}
label2id = {}
LABELS = []
with open(labels_path, "r") as f:
    LABELS = json.load(f)
    LABELS = [x[0] + ' ' + x[1] for x in LABELS]
    for i, label in enumerate(LABELS):
        id2label[i] = label
        label2id[label] = i

tokenizer = AutoTokenizer.from_pretrained("stulcrad/XLM-RoBERTa-2", add_prefix_space=True)
model = RelationExtractionModel.from_pretrained("stulcrad/XLM-RoBERTa-2", tokenizer=tokenizer,
                                                id2label=id2label, label2id=label2id,
                                                num_labels=len(LABELS))

sentence1 = "[E1] Jan [/E1] se narodil v [E2] Praze [/E2] ."
sentence2 = "Vydáno bylo 17 . února roku 2017 společností [E2] Domino Records [/E2] , téměř přesně rok po vydání desky [E1] Painting With [/E1] . "
sentence_example = "[E1] Jan Máchal [/E1] ( 9. února 1864, Třebíč [UNK] 15. října 1924, Třebíč ) byl český pedagog, činovník [E2] Sokola [/E2] . "
special_token_dict = {
    '[E1]':consts.E1_START_TOKEN, '[/E1]':consts.E1_END_TOKEN,
    '[E2]':consts.E2_START_TOKEN, '[/E2]':consts.E2_END_TOKEN
}

marked = [token if token not in special_token_dict else special_token_dict[token] for token in sentence1.split(' ')]
marked2 = [token if token not in special_token_dict else special_token_dict[token] for token in sentence2.split(' ')]
marked_example = [token if token not in special_token_dict else special_token_dict[token] for token in sentence_example.split(' ')]
# print(marked)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

marked_sentence = ""
marked_sentence_example = ""
marked_sentence2 = ""
first = True
for token in marked:
    if first:
        first = False
        marked_sentence += token
    else:
        marked_sentence += " " + token
first = True
for token in marked_example:
    if first:
        first = False
        marked_sentence_example += token
    else:
        marked_sentence_example += " " + token
first = True
for token in marked2:
    if first:
        first = False
        marked_sentence2 += token
    else:
        marked_sentence2 += " " + token


inputs = tokenizer(marked_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs_example = tokenizer(marked_sentence_example, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs2 = tokenizer(marked_sentence2, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs.to(device)
inputs_example.to(device)
inputs2.to(device)

with torch.no_grad():
    outputs = model(**inputs)
    outputs_example = model(**inputs_example)
    outputs2 = model(**inputs2)
    logits = outputs["logits"]
    logits_example = outputs_example["logits"]
    logits2 = outputs2["logits"]

pred_labels = logits.argmax().item()
pred_labels_example = logits_example.argmax().item()
pred_labels2 = logits2.argmax().item()
print(id2label[pred_labels])
print(id2label[pred_labels_example])
print(id2label[pred_labels2])
    

['P0 other', 'P19 místo narození', 'P20 místo úmrtí', 'P21 pohlaví', 'P22 otec', 'P25 matka', 'P26 choť', 'P27 státní občanství', 'P36 hlavní město', 'P39 ve funkci', 'P47 hraničí s', 'P50 autor', 'P53 rodina', 'P54 člen sportovního družstva', 'P97 šlechtický titul', 'P101 zaměření', 'P102 stranická příslušnost', 'P105 taxonomické zařazení', 'P106 povolání', 'P118 liga', 'P131 nachází se v administrativní jednotce', 'P136 žánr', 'P137 operátor', 'P138 pojmenováno po', 'P140 vyznání', 'P150 nižší správní celky', 'P155 předchozí', 'P159 sídlo', 'P161 hraje', 'P171 nadřazený taxon', 'P175 interpret', 'P176 výrobce', 'P179 série', 'P206 u vodní plochy', 'P264 hudební vydavatelství', 'P276 místo', 'P279 nadtřída', 'P360 seznam (čeho)', 'P361 část (čeho)', 'P413 pozice hráče', 'P425 obor tohoto povolání', 'P460 údajně totéž co', 'P461 protiklad', 'P463 člen (čeho)', 'P495 země původu', 'P527 skládá se z', 'P530 diplomatický vztah', 'P641 sport', 'P669 ulice', 'P706 oblast', 'P708 diecéze', '

Some weights of XLMRobertaModel were not initialized from the model checkpoint at stulcrad/XLM-RoBERTa-2 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'enco

P19 místo narození
P463 člen (čeho)
P264 hudební vydavatelství


: 